In [1]:
!pip install torch torchvision torchaudio
!pip install ogb
!pip install torch-geometric

In [ ]:
import os, random, numpy as np, torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_networkx
import networkx as nx
import wandb

wandb.login(key="Your-api-key")
Dataset_name = "ENZYMES"

# Learnable Root-Based BFS Positional Encoding GCN

In [3]:
import os, random, numpy as np, torch
import torch.nn.functional as F
from torch import nn
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_undirected, degree
import wandb
from collections import deque

EPOCHS = int(os.getenv("EPOCHS", "50"))
NUM_SEEDS = int(os.getenv("NUM_SEEDS", "30"))
PROJECT = f"GCN Graph classification({Dataset_name})_Gal"
ENTITY = "noynetanel1-tel-aviv-university"

def set_seed(s):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def prep_graph(d):
    d = d.clone()
    if d.edge_index.numel() > 0:
        d.edge_index = to_undirected(d.edge_index, num_nodes=d.num_nodes)
    if d.x is None:
        d.x = torch.ones((d.num_nodes, 1), dtype=torch.float)
    deg = degree(d.edge_index[0], d.num_nodes, dtype=torch.float) if d.edge_index.numel() > 0 else torch.zeros(d.num_nodes, dtype=torch.float)
    m = max(deg.max().item(), 1.0)
    deg = (deg / m).unsqueeze(1)
    d.x = torch.cat([d.x, deg], dim=1)
    return d

def bfs_rank_from_root(num_nodes, edge_index_local, root_local):
    adj = [[] for _ in range(num_nodes)]
    u, v = edge_index_local
    for i in range(u.numel()):
        a = int(u[i])
        b = int(v[i])
        if a != b:
            adj[a].append(b)
            adj[b].append(a)
    ranks = torch.full((num_nodes,), -1.0)
    if num_nodes == 0 or root_local < 0 or root_local >= num_nodes:
        return ranks
    q = deque([root_local])
    seen = [False] * num_nodes
    seen[root_local] = True
    order = []
    while q:
        cur = q.popleft()
        order.append(cur)
        for w in adj[cur]:
            if not seen[w]:
                seen[w] = True
                q.append(w)
    L = max(1, len(order))
    for i, node in enumerate(order):
        ranks[node] = float(i) / float(L)
    return ranks

class LearnableBFS_GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, bfs_dim=8, tau=1.0):
        super().__init__()
        self.tau = tau
        self.root_scorer = nn.Sequential(
            nn.Linear(in_channels, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
        self.bfs_proj = nn.Sequential(
            nn.Linear(1, 16),
            nn.ReLU(),
            nn.Linear(16, bfs_dim)
        )
        self.conv1 = GCNConv(in_channels + bfs_dim, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = nn.Linear(hidden_channels, out_channels)

    @torch.no_grad()
    def _compute_bfs_rank_batch(self, x, edge_index, batch, root_logits):
        device = x.device
        N = x.size(0)
        ranks = torch.full((N,), -1.0, device='cpu')
        edge_index_cpu = edge_index.detach().cpu()
        batch_cpu = batch.detach().cpu()
        logits_cpu = root_logits.detach().cpu().squeeze(-1)
        num_graphs = int(batch_cpu.max().item()) + 1 if N > 0 else 0
        for g in range(num_graphs):
            idx = (batch_cpu == g).nonzero(as_tuple=False).view(-1)
            n = idx.numel()
            if n == 0:
                continue
            logits_g = logits_cpu[idx]
            probs_g = F.gumbel_softmax(logits_g, tau=self.tau, hard=True)
            root_local = int(probs_g.argmax().item())
            map_gl2lc = torch.full((N,), -1, dtype=torch.long)
            map_gl2lc[idx] = torch.arange(n, dtype=torch.long)
            u_all, v_all = edge_index_cpu
            m_u = map_gl2lc[u_all]
            m_v = map_gl2lc[v_all]
            mask = (m_u >= 0) & (m_v >= 0)
            u_loc = m_u[mask]
            v_loc = m_v[mask]
            ranks_g = bfs_rank_from_root(n, (u_loc, v_loc), root_local)
            ranks[idx] = ranks_g
        return ranks.to(device)

    def forward(self, x, edge_index, batch):
        root_logits = self.root_scorer(x)
        bfs_rank = self._compute_bfs_rank_batch(x, edge_index, batch, root_logits)
        bfs_feat = self.bfs_proj(bfs_rank.unsqueeze(1))
        x = torch.cat([x, bfs_feat], dim=1)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        return F.log_softmax(self.lin(x), dim=1)

def run_seed(seed):
    set_seed(seed)
    dataset = TUDataset(root='data/TUDataset', name=Dataset_name, transform=prep_graph)
    if dataset.num_features == 0:
        for data in dataset:
            data.x = torch.ones((data.num_nodes, 1))
    dataset = dataset.shuffle()
    num_graphs = len(dataset)
    train_size = int(0.8 * num_graphs)
    train_dataset = dataset[:train_size]
    test_dataset = dataset[train_size:]
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    in_channels = dataset[0].x.size(1)
    model = LearnableBFS_GCN(in_channels, 64, dataset.num_classes, bfs_dim=8, tau=1.0).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    wandb.init(
        project=PROJECT,
        entity=ENTITY,
        name=f"GCN_LearnableBFS_STG_seed_{seed}",
        group=f"GCN_{Dataset_name}_LearnableBFS",
        job_type="learnable_bfs",
        reinit=True,
        config={
            "model": "GCN",
            "hidden_channels": 64,
            "epochs": EPOCHS,
            "lr": 0.001,
            "dataset": Dataset_name,
            "task": "Learnable BFS",
            "positional_encoding": "Learned root (ST-Gumbel) + BFS rank MLP",
            "seed": seed
        }
    )

    def train():
        model.train()
        total_loss = 0.0
        correct = 0
        total_samples = 0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data.x, data.edge_index, data.batch)
            loss = F.nll_loss(out, data.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
            total_samples += data.num_graphs
        train_acc = correct / total_samples
        return total_loss / len(train_loader), train_acc

    def test(loader):
        model.eval()
        correct = 0
        total_samples = 0
        for data in loader:
            data = data.to(device)
            out = model(data.x, data.edge_index, data.batch)
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
            total_samples += data.num_graphs
        return correct / total_samples

    for epoch in range(1, EPOCHS + 1):
        loss, train_acc = train()
        test_acc = test(test_loader)
        print(f"Seed {seed} | Epoch {epoch:03d} | Loss: {loss:.4f} | Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f}")
        wandb.log({"epoch": epoch, "loss": loss, "train_acc": train_acc, "test_acc": test_acc})

    wandb.finish()

if __name__ == "__main__":
    wandb.login()
    for s in range(1, NUM_SEEDS + 1):
        run_seed(s)


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


Seed 1 | Epoch 001 | Loss: 1.7972 | Train Acc: 0.1646 | Test Acc: 0.1500
Seed 1 | Epoch 002 | Loss: 1.7868 | Train Acc: 0.2083 | Test Acc: 0.1667
Seed 1 | Epoch 003 | Loss: 1.7847 | Train Acc: 0.2000 | Test Acc: 0.1917
Seed 1 | Epoch 004 | Loss: 1.7819 | Train Acc: 0.2062 | Test Acc: 0.2000
Seed 1 | Epoch 005 | Loss: 1.7787 | Train Acc: 0.1625 | Test Acc: 0.1833
Seed 1 | Epoch 006 | Loss: 1.7758 | Train Acc: 0.2104 | Test Acc: 0.2083
Seed 1 | Epoch 007 | Loss: 1.7708 | Train Acc: 0.2167 | Test Acc: 0.2500
Seed 1 | Epoch 008 | Loss: 1.7667 | Train Acc: 0.1938 | Test Acc: 0.2167
Seed 1 | Epoch 009 | Loss: 1.7657 | Train Acc: 0.2271 | Test Acc: 0.2333
Seed 1 | Epoch 010 | Loss: 1.7599 | Train Acc: 0.2396 | Test Acc: 0.2667
Seed 1 | Epoch 011 | Loss: 1.7602 | Train Acc: 0.2042 | Test Acc: 0.1917
Seed 1 | Epoch 012 | Loss: 1.7546 | Train Acc: 0.2313 | Test Acc: 0.2917
Seed 1 | Epoch 013 | Loss: 1.7569 | Train Acc: 0.2750 | Test Acc: 0.2833
Seed 1 | Epoch 014 | Loss: 1.7511 | Train Acc: 0.26

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▁▂▁▂▂▂▁
test_acc,▁▂▃▃▃▆▄▅▆▃▇█▆█▅▇▅▄▇▆▆█▅▆▆▃▅▄▆▅▇▆▄▅▅▆▅▄▆▆
train_acc,▁▄▃▁▄▃▅▅▃▅▇▆▆▆▃▆▇▆▅▆▇▇▆▇▆▇▆▆▇▇█▇▇▆▇█▇▆▇▇
epoch,50
loss,1.68387
test_acc,0.26667
train_acc,0.27292


Seed 2 | Epoch 001 | Loss: 1.7971 | Train Acc: 0.1646 | Test Acc: 0.1500
Seed 2 | Epoch 002 | Loss: 1.7904 | Train Acc: 0.2104 | Test Acc: 0.1917
Seed 2 | Epoch 003 | Loss: 1.7843 | Train Acc: 0.2396 | Test Acc: 0.1917
Seed 2 | Epoch 004 | Loss: 1.7818 | Train Acc: 0.2271 | Test Acc: 0.1917
Seed 2 | Epoch 005 | Loss: 1.7776 | Train Acc: 0.2354 | Test Acc: 0.2333
Seed 2 | Epoch 006 | Loss: 1.7731 | Train Acc: 0.2458 | Test Acc: 0.1917
Seed 2 | Epoch 007 | Loss: 1.7685 | Train Acc: 0.2167 | Test Acc: 0.2167
Seed 2 | Epoch 008 | Loss: 1.7636 | Train Acc: 0.2167 | Test Acc: 0.2000
Seed 2 | Epoch 009 | Loss: 1.7598 | Train Acc: 0.2167 | Test Acc: 0.1667
Seed 2 | Epoch 010 | Loss: 1.7568 | Train Acc: 0.2208 | Test Acc: 0.2167
Seed 2 | Epoch 011 | Loss: 1.7560 | Train Acc: 0.2208 | Test Acc: 0.2333
Seed 2 | Epoch 012 | Loss: 1.7503 | Train Acc: 0.2167 | Test Acc: 0.1750
Seed 2 | Epoch 013 | Loss: 1.7498 | Train Acc: 0.2437 | Test Acc: 0.2333
Seed 2 | Epoch 014 | Loss: 1.7485 | Train Acc: 0.24

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,██▇▇▇▆▆▅▅▅▅▄▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▂▁▁▁▁
test_acc,▁▃▃▃▃▃▂▄▂▄▃▅▄▄▅▅▅▅▆▅▆▆▆▅▆▆▆▆▆▆▇▆▇▇▆▆█▇▆▇
train_acc,▁▃▅▄▅▄▄▄▄▄▅▅▄▅▅▆▆▆▆▆▆▆▇▇▇▇▇█▇▇▇▇▇▇▇▇█▇▇█
epoch,50
loss,1.7
test_acc,0.30833
train_acc,0.3


...

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇███
loss,██▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▂▁▁▁
test_acc,▂▁▅▆▃▅▆▄▄▅▄▆▇▅▅▆█▆▇▆▇▆▆▆▇█▇▇▇██▆█▅██▆▇▇█
train_acc,▁▂▃▄▄▃▆▄▆▆▆▆▆▅▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██▇▇█▇
epoch,50
loss,1.65609
test_acc,0.24167
train_acc,0.30833


Seed 29 | Epoch 001 | Loss: 1.7945 | Train Acc: 0.1500 | Test Acc: 0.1333
Seed 29 | Epoch 002 | Loss: 1.7884 | Train Acc: 0.2000 | Test Acc: 0.2250
Seed 29 | Epoch 003 | Loss: 1.7861 | Train Acc: 0.2417 | Test Acc: 0.2167
Seed 29 | Epoch 004 | Loss: 1.7843 | Train Acc: 0.2167 | Test Acc: 0.2083
Seed 29 | Epoch 005 | Loss: 1.7817 | Train Acc: 0.2208 | Test Acc: 0.2417
Seed 29 | Epoch 006 | Loss: 1.7788 | Train Acc: 0.2167 | Test Acc: 0.2000
Seed 29 | Epoch 007 | Loss: 1.7762 | Train Acc: 0.2042 | Test Acc: 0.1833
Seed 29 | Epoch 008 | Loss: 1.7742 | Train Acc: 0.2313 | Test Acc: 0.2250
Seed 29 | Epoch 009 | Loss: 1.7678 | Train Acc: 0.2500 | Test Acc: 0.2000
Seed 29 | Epoch 010 | Loss: 1.7657 | Train Acc: 0.2167 | Test Acc: 0.1917
Seed 29 | Epoch 011 | Loss: 1.7620 | Train Acc: 0.2521 | Test Acc: 0.2417
Seed 29 | Epoch 012 | Loss: 1.7617 | Train Acc: 0.2250 | Test Acc: 0.1833
Seed 29 | Epoch 013 | Loss: 1.7572 | Train Acc: 0.2333 | Test Acc: 0.2417
Seed 29 | Epoch 014 | Loss: 1.7526 | T

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,██▇▇▇▇▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁
test_acc,▄▃▃▅▂▄▂▂▅▁▃▅▆▄▄▄▄▅▅▄▅▅▄▆▆▅▆▅▆▅▆▅▇▇▅▅▆█▅▆
train_acc,▁▃▅▄▄▄▅▆▄▆▅▅▆▅▆▇▇▇█▇▇▇▇█▇█▇▇▇█▇███████▇█
epoch,50
loss,1.68527
test_acc,0.25833
train_acc,0.29583


Seed 30 | Epoch 001 | Loss: 1.7987 | Train Acc: 0.1750 | Test Acc: 0.1250
Seed 30 | Epoch 002 | Loss: 1.7868 | Train Acc: 0.1750 | Test Acc: 0.1250
Seed 30 | Epoch 003 | Loss: 1.7860 | Train Acc: 0.1812 | Test Acc: 0.1167
Seed 30 | Epoch 004 | Loss: 1.7824 | Train Acc: 0.2104 | Test Acc: 0.2250
Seed 30 | Epoch 005 | Loss: 1.7815 | Train Acc: 0.1938 | Test Acc: 0.1500
Seed 30 | Epoch 006 | Loss: 1.7776 | Train Acc: 0.2167 | Test Acc: 0.1333
Seed 30 | Epoch 007 | Loss: 1.7761 | Train Acc: 0.1812 | Test Acc: 0.1167
Seed 30 | Epoch 008 | Loss: 1.7721 | Train Acc: 0.1771 | Test Acc: 0.1667
Seed 30 | Epoch 009 | Loss: 1.7701 | Train Acc: 0.1979 | Test Acc: 0.2000
Seed 30 | Epoch 010 | Loss: 1.7664 | Train Acc: 0.1896 | Test Acc: 0.1583
Seed 30 | Epoch 011 | Loss: 1.7654 | Train Acc: 0.2062 | Test Acc: 0.1833
Seed 30 | Epoch 012 | Loss: 1.7624 | Train Acc: 0.2313 | Test Acc: 0.2000
Seed 30 | Epoch 013 | Loss: 1.7611 | Train Acc: 0.2271 | Test Acc: 0.1583
Seed 30 | Epoch 014 | Loss: 1.7596 | T

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▇▇▆▆▆▅▅▅▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▂▃▂▂▂▂▂▁▂▁▂▁▁
test_acc,▁▁▁▆▂▁▃▅▃▄▃▄▄▄▄▇▄▄▅▆▅▆▇▅▅▇▄▄▄▇▇▆▇█▇█▇▆▇▇
train_acc,▁▁▁▄▂▁▁▃▂▃▅▁▂▅▂▃▆▅▄▃▅▃▄▆▅▅▄▅▆▆█▆▆▇▇▆▇▇▇▇
epoch,50
loss,1.70823
test_acc,0.25833
train_acc,0.25625


# **Layerwise BFS**

In [4]:
# **Layerwise BFS for Protein**
import os, random, numpy as np, torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_networkx
import networkx as nx
import wandb

EPOCHS = int(os.getenv("EPOCHS", "50"))
NUM_SEEDS = int(os.getenv("NUM_SEEDS", "30"))
PROJECT = f"GCN Graph classification({Dataset_name})_Gal"
ENTITY = "noynetanel1-tel-aviv-university"

def set_seed(s):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, bfs_dim=8):
        super().__init__()
        self.bfs_proj = torch.nn.Sequential(
            torch.nn.Linear(1, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, bfs_dim)
        )
        self.conv1 = GCNConv(in_channels + bfs_dim, hidden_channels)
        self.conv2 = GCNConv(hidden_channels + bfs_dim, hidden_channels)
        self.lin = torch.nn.Linear(hidden_channels, out_channels)

    def forward(self, x, bfs_rank, edge_index, batch):
        bfs_feat = self.bfs_proj(bfs_rank.unsqueeze(1))
        x = torch.cat([x, bfs_feat], dim=1)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        bfs_feat2 = self.bfs_proj(bfs_rank.unsqueeze(1))
        x = torch.cat([x, bfs_feat2], dim=1)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        return F.log_softmax(self.lin(x), dim=1)

def make_bfs_rank_transform():
    def transform(data):
        G = to_networkx(data, to_undirected=True)
        bfs_rank = torch.full((data.num_nodes,), -1.0, dtype=torch.float)
        if G.number_of_nodes() > 0:
            degrees = dict(G.degree())
            root = max(degrees, key=degrees.get) if degrees else 0
            bfs_order = list(nx.bfs_tree(G, root))
            L = max(1, len(bfs_order))
            for i, node in enumerate(bfs_order):
                bfs_rank[node] = i / L
        data.bfs_rank = bfs_rank
        # REDDIT-BINARY has no node features; use degree as feature or 1s
        if not hasattr(data, 'x') or data.x is None:
            deg = torch.tensor([d for _, d in G.degree()], dtype=torch.float).unsqueeze(1)
            data.x = deg
        return data
    return transform

def run_seed(SEED):
    set_seed(SEED)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    dataset = TUDataset(root='data/TUDataset', name=Dataset_name, transform=make_bfs_rank_transform())
    torch.manual_seed(SEED)
    dataset = dataset.shuffle()
    train_dataset = dataset[:len(dataset)//2]
    test_dataset = dataset[len(dataset)//2:]
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

    wandb.init(
        project=PROJECT,
        entity=ENTITY,
        name=f"GCN_Layerwise_BFS_{Dataset_name}_seed_{SEED}",
        group=f"GCN_{Dataset_name}_Layerwise_BFS",
        job_type="layerwise-bfs",
        reinit=True,
        config={
            "model": "GCN",
            "hidden_channels": 64,
            "epochs": EPOCHS,
            "lr": 0.001,
            "dataset": Dataset_name,
            "task": "Layerwise_BFS_Injection",
            "seed": SEED,
            "cuda_available": torch.cuda.is_available(),
            "gpu_name": torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU",
            "cuda_version": torch.version.cuda if torch.cuda.is_available() else None
        }
    )

    model = GCN(
        in_channels=dataset.num_features,
        hidden_channels=64,
        out_channels=dataset.num_classes
    ).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    def train():
        model.train()
        total_loss = 0.0
        correct = 0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data.x, data.bfs_rank, data.edge_index, data.batch)
            loss = F.nll_loss(out, data.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * data.num_graphs
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
        acc = correct / len(train_dataset)
        return total_loss / len(train_dataset), acc
    
    def test():
        model.eval()
        correct = 0
        for data in test_loader:
            data = data.to(device)
            out = model(data.x, data.bfs_rank, data.edge_index, data.batch)
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
        return correct / len(test_dataset)
    
    for epoch in range(1, EPOCHS + 1):
        loss, train_acc = train()
        test_acc = test()
        print(f"Seed {SEED} | Epoch {epoch:03d} | Loss: {loss:.4f} | Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f}")
        wandb.log({"epoch": epoch, "loss": loss, "train_acc": train_acc, "test_acc": test_acc})
    wandb.finish()

if __name__ == "__main__":
    wandb.login()
    for s in range(1, NUM_SEEDS + 1):
        run_seed(s)


Seed 1 | Epoch 001 | Loss: 1.8012 | Train Acc: 0.1700 | Test Acc: 0.1700
Seed 1 | Epoch 002 | Loss: 1.7887 | Train Acc: 0.1667 | Test Acc: 0.1400
Seed 1 | Epoch 003 | Loss: 1.7836 | Train Acc: 0.2000 | Test Acc: 0.1367
Seed 1 | Epoch 004 | Loss: 1.7800 | Train Acc: 0.2033 | Test Acc: 0.1367
Seed 1 | Epoch 005 | Loss: 1.7761 | Train Acc: 0.2233 | Test Acc: 0.1733
Seed 1 | Epoch 006 | Loss: 1.7758 | Train Acc: 0.2200 | Test Acc: 0.1667
Seed 1 | Epoch 007 | Loss: 1.7697 | Train Acc: 0.2067 | Test Acc: 0.1367
Seed 1 | Epoch 008 | Loss: 1.7640 | Train Acc: 0.2167 | Test Acc: 0.1333
Seed 1 | Epoch 009 | Loss: 1.7632 | Train Acc: 0.2367 | Test Acc: 0.1700
Seed 1 | Epoch 010 | Loss: 1.7562 | Train Acc: 0.2200 | Test Acc: 0.1500
Seed 1 | Epoch 011 | Loss: 1.7532 | Train Acc: 0.2000 | Test Acc: 0.1333
Seed 1 | Epoch 012 | Loss: 1.7499 | Train Acc: 0.2133 | Test Acc: 0.1367
Seed 1 | Epoch 013 | Loss: 1.7463 | Train Acc: 0.2033 | Test Acc: 0.1467
Seed 1 | Epoch 014 | Loss: 1.7453 | Train Acc: 0.19

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
loss,█▇▇▇▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁
test_acc,▁▁▁▄▃▁▃▂▁▁▃▇▂▁▅▃▂▂▄▄▄▅▃▆▆▅▆▃▃▇▆▅▄█▄▄▅▇▃▄
train_acc,▁▁▃▃▅▄▄▆▅▃▃▃▆▇▂▇▆▅▅▆▆▄▆▆▅▅▇▆▇█▅▅▅▇█▅▆▆▆▃
epoch,50
loss,1.70478
test_acc,0.18333
train_acc,0.20333


Seed 2 | Epoch 001 | Loss: 1.8087 | Train Acc: 0.1533 | Test Acc: 0.1833
Seed 2 | Epoch 002 | Loss: 1.7938 | Train Acc: 0.1467 | Test Acc: 0.1567
Seed 2 | Epoch 003 | Loss: 1.7880 | Train Acc: 0.2100 | Test Acc: 0.1567
Seed 2 | Epoch 004 | Loss: 1.7868 | Train Acc: 0.2033 | Test Acc: 0.1533
Seed 2 | Epoch 005 | Loss: 1.7839 | Train Acc: 0.1967 | Test Acc: 0.2033
Seed 2 | Epoch 006 | Loss: 1.7815 | Train Acc: 0.2067 | Test Acc: 0.1967
Seed 2 | Epoch 007 | Loss: 1.7789 | Train Acc: 0.2100 | Test Acc: 0.2433
Seed 2 | Epoch 008 | Loss: 1.7769 | Train Acc: 0.2100 | Test Acc: 0.2367
Seed 2 | Epoch 009 | Loss: 1.7719 | Train Acc: 0.2500 | Test Acc: 0.2033
Seed 2 | Epoch 010 | Loss: 1.7699 | Train Acc: 0.2567 | Test Acc: 0.1933
Seed 2 | Epoch 011 | Loss: 1.7667 | Train Acc: 0.2533 | Test Acc: 0.2000
Seed 2 | Epoch 012 | Loss: 1.7628 | Train Acc: 0.2333 | Test Acc: 0.1967
Seed 2 | Epoch 013 | Loss: 1.7603 | Train Acc: 0.2633 | Test Acc: 0.2300
Seed 2 | Epoch 014 | Loss: 1.7558 | Train Acc: 0.25

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▇▇▆▆▆▆▆▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁
test_acc,▃▁▁▁▄▆▅▄▃▃▅▅▄▅▃▆▄▃▅▅▄▆▅▆▅▇▆▄▇█▅▇▅▅▇▆▆▆█▆
train_acc,▁▁▄▄▃▄▄▆▆▆▆▇▇▆▅▆▄▃▇▆▇▆▆▇▇█▇▇▆▇▇██▇█▇▇█▇▆
epoch,50
loss,1.68776
test_acc,0.26667
train_acc,0.26667


Seed 3 | Epoch 001 | Loss: 1.7972 | Train Acc: 0.1867 | Test Acc: 0.1700
Seed 3 | Epoch 002 | Loss: 1.7885 | Train Acc: 0.1833 | Test Acc: 0.1567
Seed 3 | Epoch 003 | Loss: 1.7876 | Train Acc: 0.1867 | Test Acc: 0.1700
Seed 3 | Epoch 004 | Loss: 1.7849 | Train Acc: 0.1767 | Test Acc: 0.1867
Seed 3 | Epoch 005 | Loss: 1.7830 | Train Acc: 0.1800 | Test Acc: 0.1900
Seed 3 | Epoch 006 | Loss: 1.7799 | Train Acc: 0.2000 | Test Acc: 0.1767
Seed 3 | Epoch 007 | Loss: 1.7776 | Train Acc: 0.1867 | Test Acc: 0.1833
Seed 3 | Epoch 008 | Loss: 1.7757 | Train Acc: 0.1933 | Test Acc: 0.1933
Seed 3 | Epoch 009 | Loss: 1.7745 | Train Acc: 0.2067 | Test Acc: 0.2200
Seed 3 | Epoch 010 | Loss: 1.7693 | Train Acc: 0.2333 | Test Acc: 0.2033
Seed 3 | Epoch 011 | Loss: 1.7700 | Train Acc: 0.2500 | Test Acc: 0.2100
Seed 3 | Epoch 012 | Loss: 1.7669 | Train Acc: 0.2167 | Test Acc: 0.2067
Seed 3 | Epoch 013 | Loss: 1.7635 | Train Acc: 0.2233 | Test Acc: 0.2067
Seed 3 | Epoch 014 | Loss: 1.7607 | Train Acc: 0.21

...

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
loss,█▇▇▇▇▆▆▆▆▅▅▅▄▄▄▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁
test_acc,▁▁▁▁▄▃▃▃▅▄▃▃▃▄▄▅▃▄▅▃▆▄▃▃▃▅▇█▅▆▆▅▆▆▇▇▇▆▄▇
train_acc,▂▃▁▁▄▅▅▅▄▇▆▆▅▄▅▆▆▅▆▅▅▆▆▅▃▆█▆▆▆▇▇█▇▆▇▇▇▆▆
epoch,50
loss,1.69139
test_acc,0.25
train_acc,0.26667


Seed 29 | Epoch 001 | Loss: 1.8046 | Train Acc: 0.1533 | Test Acc: 0.1667
Seed 29 | Epoch 002 | Loss: 1.7934 | Train Acc: 0.1833 | Test Acc: 0.1600
Seed 29 | Epoch 003 | Loss: 1.7906 | Train Acc: 0.1833 | Test Acc: 0.1300
Seed 29 | Epoch 004 | Loss: 1.7892 | Train Acc: 0.1567 | Test Acc: 0.1367
Seed 29 | Epoch 005 | Loss: 1.7870 | Train Acc: 0.1500 | Test Acc: 0.1400
Seed 29 | Epoch 006 | Loss: 1.7868 | Train Acc: 0.1567 | Test Acc: 0.1333
Seed 29 | Epoch 007 | Loss: 1.7848 | Train Acc: 0.1367 | Test Acc: 0.1300
Seed 29 | Epoch 008 | Loss: 1.7821 | Train Acc: 0.1633 | Test Acc: 0.1700
Seed 29 | Epoch 009 | Loss: 1.7818 | Train Acc: 0.1700 | Test Acc: 0.1300
Seed 29 | Epoch 010 | Loss: 1.7796 | Train Acc: 0.1700 | Test Acc: 0.1733
Seed 29 | Epoch 011 | Loss: 1.7767 | Train Acc: 0.2067 | Test Acc: 0.1500
Seed 29 | Epoch 012 | Loss: 1.7744 | Train Acc: 0.1633 | Test Acc: 0.1467
Seed 29 | Epoch 013 | Loss: 1.7716 | Train Acc: 0.1533 | Test Acc: 0.1400
Seed 29 | Epoch 014 | Loss: 1.7714 | T

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁
test_acc,▃▃▁▁▂▁▃▁▃▂▂▃▂▅▃▅▇▄▅▅██▆▄▄▆▇▇▇▇▇▇▆▇▆▇▇█▇▇
train_acc,▂▂▂▂▁▃▃▅▃▂▃▃▅▃▅▅▄▄▅▆▆▇▇▆▄▅▇▇▇▅▆▇▇▆▅▇▆▆▆█
epoch,50
loss,1.71869
test_acc,0.25
train_acc,0.26


Seed 30 | Epoch 001 | Loss: 1.8027 | Train Acc: 0.1333 | Test Acc: 0.1400
Seed 30 | Epoch 002 | Loss: 1.7855 | Train Acc: 0.1800 | Test Acc: 0.1533
Seed 30 | Epoch 003 | Loss: 1.7824 | Train Acc: 0.1933 | Test Acc: 0.1800
Seed 30 | Epoch 004 | Loss: 1.7817 | Train Acc: 0.1800 | Test Acc: 0.1300
Seed 30 | Epoch 005 | Loss: 1.7803 | Train Acc: 0.1933 | Test Acc: 0.1867
Seed 30 | Epoch 006 | Loss: 1.7780 | Train Acc: 0.2133 | Test Acc: 0.1467
Seed 30 | Epoch 007 | Loss: 1.7757 | Train Acc: 0.2233 | Test Acc: 0.2300
Seed 30 | Epoch 008 | Loss: 1.7722 | Train Acc: 0.2500 | Test Acc: 0.1800
Seed 30 | Epoch 009 | Loss: 1.7704 | Train Acc: 0.2233 | Test Acc: 0.1800
Seed 30 | Epoch 010 | Loss: 1.7680 | Train Acc: 0.2433 | Test Acc: 0.1767
Seed 30 | Epoch 011 | Loss: 1.7680 | Train Acc: 0.2033 | Test Acc: 0.1833
Seed 30 | Epoch 012 | Loss: 1.7620 | Train Acc: 0.2100 | Test Acc: 0.1467
Seed 30 | Epoch 013 | Loss: 1.7606 | Train Acc: 0.2167 | Test Acc: 0.1800
Seed 30 | Epoch 014 | Loss: 1.7590 | T

epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▇▇▆▆▆▆▆▅▅▅▄▅▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁
test_acc,▁▂▄▅▂▄▄▄▄▂▅▃▃▃▄▃▄▅▄▄▅▃▃▂▅▆▆▆▇▆▃▆▆▆▆▇▇▇▄█
train_acc,▁▃▄▃▄▆▇▆▅▅▆▅▅▅▅▅▅▅▅▆▆▅▅▅▆▆▇▇▇▇▇▆██▇▇██▇█
epoch,50
loss,1.70398
test_acc,0.23
train_acc,0.26333


# **GCN Learnable BFS Sinusoidal**

In [5]:
import os, random, numpy as np, torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_networkx
import networkx as nx
import math
import wandb

EPOCHS = int(os.getenv("EPOCHS", "50"))
NUM_SEEDS = int(os.getenv("NUM_SEEDS", "30"))
PROJECT = f"GCN Graph classification({Dataset_name})_Gal"
ENTITY = "noynetanel1-tel-aviv-university"

def set_seed(s):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

class LearnableBFSPositionalEncoding(nn.Module):
    def __init__(self, dim, max_len=512):
        super().__init__()
        self.lin = nn.Linear(1, dim)
        pe = torch.zeros(max_len, dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, dim, 2).float() * (-math.log(10000.0) / dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer("pe", pe)
        self.max_len = max_len
    def forward(self, bfs_rank):
        idx = (bfs_rank.clamp(0, 1) * (self.max_len - 1)).long()
        return self.lin(bfs_rank.unsqueeze(1)) + self.pe[idx]

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, bfs_dim=32, max_len=512):
        super().__init__()
        self.bfs_enc = LearnableBFSPositionalEncoding(bfs_dim, max_len)
        self.conv1 = GCNConv(in_channels + bfs_dim, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = nn.Linear(hidden_channels, out_channels)
    def forward(self, x, bfs_rank, edge_index, batch):
        x = torch.cat([x, self.bfs_enc(bfs_rank)], dim=1)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        return F.log_softmax(self.lin(x), dim=1)

def make_bfs_rank_transform():
    def transform(data):
        G = to_networkx(data, to_undirected=True)
        bfs_rank = torch.full((data.num_nodes,), -1.0, dtype=torch.float)
        if G.number_of_nodes() > 0:
            deg = dict(G.degree())
            root = max(deg, key=deg.get) if len(deg) > 0 else 0
            order = list(nx.bfs_tree(G, root))
            L = max(1, len(order))
            for i, node in enumerate(order):
                bfs_rank[node] = i / L
        data.bfs_rank = bfs_rank
        return data
    return transform
    
def run_seed(SEED):
    set_seed(SEED)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    dataset = TUDataset(root="data/TUDataset", name=Dataset_name, transform=make_bfs_rank_transform())
    torch.manual_seed(SEED)
    dataset = dataset.shuffle()
    num_graphs = len(dataset)
    train_size = int(0.8 * num_graphs)  # 80% for train
    train_dataset = dataset[:train_size]
    test_dataset = dataset[train_size:]
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

    wandb.init(
        project=PROJECT,
        entity=ENTITY,
        name=f"GCN_LearnableSinBFS_seed_{SEED}",
        group=f"GCN_{Dataset_name}_LearnableSinBFS",
        job_type="learnable-sin-bfs",
        reinit=True,
        config={
            "model": "GCN",
            "hidden_channels": 64,
            "epochs": EPOCHS,
            "lr": 0.001,
            "dataset": Dataset_name,
            "task": "Learnable_Sinusoidal_BFS",
            "seed": SEED,
            "cuda_available": torch.cuda.is_available(),
            "gpu_name": torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU",
            "cuda_version": torch.version.cuda if torch.cuda.is_available() else None
        }
    )

    model = GCN(
        in_channels=dataset.num_features,
        hidden_channels=64,
        out_channels=dataset.num_classes,
        bfs_dim=32,
        max_len=512
    ).to(device)

    opt = torch.optim.Adam(model.parameters(), lr=0.001)

    def train():
        model.train()
        total_loss = 0.0
        correct = 0
        total = 0
        for data in train_loader:
            data = data.to(device)
            out = model(data.x, data.bfs_rank, data.edge_index, data.batch)
            loss = F.nll_loss(out, data.y)
            opt.zero_grad()
            loss.backward()
            opt.step()
            total_loss += loss.item() * data.num_graphs
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
            total += data.num_graphs
        return total_loss / len(train_dataset), correct / total

    def test():
        model.eval()
        correct = 0
        for data in test_loader:
            data = data.to(device)
            out = model(data.x, data.bfs_rank, data.edge_index, data.batch)
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
        return correct / len(test_dataset)

    for epoch in range(1, EPOCHS + 1):
        loss, train_acc = train()
        test_acc = test()
        print(f"Seed {SEED} | Epoch {epoch:03d} | Loss: {loss:.4f} | Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f}")
        wandb.log({"epoch": epoch, "loss": loss, "train_acc": train_acc, "test_acc": test_acc})

    wandb.finish()

if __name__ == "__main__":
    wandb.login()
    for s in range(1, NUM_SEEDS + 1):
        run_seed(s)

Seed 1 | Epoch 001 | Loss: 1.8108 | Train Acc: 0.1458 | Test Acc: 0.1917
Seed 1 | Epoch 002 | Loss: 1.7918 | Train Acc: 0.1646 | Test Acc: 0.1750
Seed 1 | Epoch 003 | Loss: 1.7877 | Train Acc: 0.1875 | Test Acc: 0.1833
Seed 1 | Epoch 004 | Loss: 1.7878 | Train Acc: 0.1729 | Test Acc: 0.1750
Seed 1 | Epoch 005 | Loss: 1.7832 | Train Acc: 0.2042 | Test Acc: 0.1833
Seed 1 | Epoch 006 | Loss: 1.7783 | Train Acc: 0.2042 | Test Acc: 0.1917
Seed 1 | Epoch 007 | Loss: 1.7750 | Train Acc: 0.2188 | Test Acc: 0.1833
Seed 1 | Epoch 008 | Loss: 1.7749 | Train Acc: 0.1958 | Test Acc: 0.1917
Seed 1 | Epoch 009 | Loss: 1.7711 | Train Acc: 0.1979 | Test Acc: 0.1667
Seed 1 | Epoch 010 | Loss: 1.7669 | Train Acc: 0.2417 | Test Acc: 0.2333
Seed 1 | Epoch 011 | Loss: 1.7658 | Train Acc: 0.2167 | Test Acc: 0.1583
Seed 1 | Epoch 012 | Loss: 1.7608 | Train Acc: 0.2167 | Test Acc: 0.2083
Seed 1 | Epoch 013 | Loss: 1.7529 | Train Acc: 0.2375 | Test Acc: 0.2083
Seed 1 | Epoch 014 | Loss: 1.7541 | Train Acc: 0.25

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▇▇▇▆▆▆▆▆▆▆▆▅▅▅▅▄▄▄▄▃▄▄▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁
test_acc,▃▂▂▂▂▂▃▁▅▁▄▄▃▆▅▄▂▂▃▄▅▅▇▂▅▅▄▇▇▆█▄▆▂▆██▄▅▇
train_acc,▁▂▃▂▃▄▃▃▅▄▅▅▄▄▅▄▄▆▄▄▆▄▅▆▆▅▆▇▅▅▇▆▇▇▇▆▇▆█▆
epoch,50
loss,1.64612
test_acc,0.26667
train_acc,0.28542


Seed 2 | Epoch 001 | Loss: 1.8122 | Train Acc: 0.1271 | Test Acc: 0.1333
Seed 2 | Epoch 002 | Loss: 1.7931 | Train Acc: 0.1854 | Test Acc: 0.1333
Seed 2 | Epoch 003 | Loss: 1.7925 | Train Acc: 0.2042 | Test Acc: 0.1750
Seed 2 | Epoch 004 | Loss: 1.7934 | Train Acc: 0.1854 | Test Acc: 0.1500
Seed 2 | Epoch 005 | Loss: 1.7850 | Train Acc: 0.2042 | Test Acc: 0.1417
Seed 2 | Epoch 006 | Loss: 1.7883 | Train Acc: 0.1917 | Test Acc: 0.1583
Seed 2 | Epoch 007 | Loss: 1.7828 | Train Acc: 0.2062 | Test Acc: 0.2000
Seed 2 | Epoch 008 | Loss: 1.7773 | Train Acc: 0.2083 | Test Acc: 0.1833
Seed 2 | Epoch 009 | Loss: 1.7759 | Train Acc: 0.2229 | Test Acc: 0.2167
Seed 2 | Epoch 010 | Loss: 1.7731 | Train Acc: 0.2208 | Test Acc: 0.2083
Seed 2 | Epoch 011 | Loss: 1.7695 | Train Acc: 0.1896 | Test Acc: 0.1833
Seed 2 | Epoch 012 | Loss: 1.7643 | Train Acc: 0.1875 | Test Acc: 0.2417
Seed 2 | Epoch 013 | Loss: 1.7642 | Train Acc: 0.2458 | Test Acc: 0.2167
Seed 2 | Epoch 014 | Loss: 1.7589 | Train Acc: 0.21

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▇▇▇▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▃▄▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁
test_acc,▁▁▃▂▁▄▃▅▄▃▅▅▅▄▄▄▄▅▅▇▄▆▆▆▅▇▆▇▅▇▇▆▅▆▅▅▆███
train_acc,▁▃▄▃▄▄▄▄▄▃▅▄▆▆▅▆▆▆▆▅▆▆▆▆▇▆▆▆▅▆▇▇▇▇▆█▇█▆▇
epoch,50
loss,1.63675
test_acc,0.29167
train_acc,0.29167


Seed 3 | Epoch 001 | Loss: 1.8069 | Train Acc: 0.1583 | Test Acc: 0.1417
Seed 3 | Epoch 002 | Loss: 1.7998 | Train Acc: 0.1458 | Test Acc: 0.1417
Seed 3 | Epoch 003 | Loss: 1.8017 | Train Acc: 0.1667 | Test Acc: 0.1083
Seed 3 | Epoch 004 | Loss: 1.7967 | Train Acc: 0.1521 | Test Acc: 0.1667
Seed 3 | Epoch 005 | Loss: 1.7903 | Train Acc: 0.1917 | Test Acc: 0.1500
Seed 3 | Epoch 006 | Loss: 1.7869 | Train Acc: 0.1917 | Test Acc: 0.1833
Seed 3 | Epoch 007 | Loss: 1.7859 | Train Acc: 0.1979 | Test Acc: 0.1667
Seed 3 | Epoch 008 | Loss: 1.7828 | Train Acc: 0.1938 | Test Acc: 0.1833
Seed 3 | Epoch 009 | Loss: 1.7858 | Train Acc: 0.1708 | Test Acc: 0.1583
Seed 3 | Epoch 010 | Loss: 1.7785 | Train Acc: 0.1979 | Test Acc: 0.1583
Seed 3 | Epoch 011 | Loss: 1.7757 | Train Acc: 0.2062 | Test Acc: 0.2167
Seed 3 | Epoch 012 | Loss: 1.7707 | Train Acc: 0.2562 | Test Acc: 0.2167
Seed 3 | Epoch 013 | Loss: 1.7770 | Train Acc: 0.2083 | Test Acc: 0.1750
Seed 3 | Epoch 014 | Loss: 1.7644 | Train Acc: 0.20

...

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
loss,█▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▄▄▄▄▄▄▃▃▃▄▃▃▃▃▂▃▂▂▂▂▂▁▁▁
test_acc,▄▁▄▁▃▃▄▃▅▅▂▂▄▂▄▄▂▄▂▁▃▅▅▁▃▄▅▄█▃▃▅▇▄▆▄▅▅█▄
train_acc,▁▁▁▁▂▃▄▃▄▃▄▆▃▄▅▆▆▄▆▄▅▄▅▆▇▅▅▆▅▆▆▆▇▇▇▇▇▇▇█
epoch,50
loss,1.62044
test_acc,0.18333
train_acc,0.36667


Seed 29 | Epoch 001 | Loss: 1.8206 | Train Acc: 0.1500 | Test Acc: 0.1500
Seed 29 | Epoch 002 | Loss: 1.7917 | Train Acc: 0.1875 | Test Acc: 0.1833
Seed 29 | Epoch 003 | Loss: 1.7914 | Train Acc: 0.1667 | Test Acc: 0.2083
Seed 29 | Epoch 004 | Loss: 1.7900 | Train Acc: 0.1771 | Test Acc: 0.1583
Seed 29 | Epoch 005 | Loss: 1.7856 | Train Acc: 0.1875 | Test Acc: 0.1750
Seed 29 | Epoch 006 | Loss: 1.7830 | Train Acc: 0.1896 | Test Acc: 0.1917
Seed 29 | Epoch 007 | Loss: 1.7799 | Train Acc: 0.2083 | Test Acc: 0.2000
Seed 29 | Epoch 008 | Loss: 1.7785 | Train Acc: 0.1938 | Test Acc: 0.2083
Seed 29 | Epoch 009 | Loss: 1.7740 | Train Acc: 0.1917 | Test Acc: 0.2000
Seed 29 | Epoch 010 | Loss: 1.7710 | Train Acc: 0.2062 | Test Acc: 0.2000
Seed 29 | Epoch 011 | Loss: 1.7674 | Train Acc: 0.2271 | Test Acc: 0.2000
Seed 29 | Epoch 012 | Loss: 1.7656 | Train Acc: 0.2250 | Test Acc: 0.1667
Seed 29 | Epoch 013 | Loss: 1.7636 | Train Acc: 0.2333 | Test Acc: 0.1583
Seed 29 | Epoch 014 | Loss: 1.7605 | T

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▅▅▅▄▄▄▃▃▃▃▃▄▃▃▂▃▂▂▂▁▁▁▁▁
test_acc,▁▄▂▃▄▆▅▅▅▂▇▇█▂▄▂▇▄▇▄▃▅▄▆▇▇▂▄▄▃▅▇▇▇▅▇█▆▇▇
train_acc,▁▂▂▂▂▃▃▃▄▄▅▄▄▅▅▄▄▃▅▆▅▆▆▅▆▆▆▅▆▆▆▇▅▆▇▇▇█▇█
epoch,50
loss,1.65324
test_acc,0.21667
train_acc,0.33542


Seed 30 | Epoch 001 | Loss: 1.8255 | Train Acc: 0.1583 | Test Acc: 0.1583
Seed 30 | Epoch 002 | Loss: 1.7981 | Train Acc: 0.1958 | Test Acc: 0.1333
Seed 30 | Epoch 003 | Loss: 1.7875 | Train Acc: 0.1833 | Test Acc: 0.1083
Seed 30 | Epoch 004 | Loss: 1.7896 | Train Acc: 0.1562 | Test Acc: 0.1000
Seed 30 | Epoch 005 | Loss: 1.7849 | Train Acc: 0.1917 | Test Acc: 0.1750
Seed 30 | Epoch 006 | Loss: 1.7822 | Train Acc: 0.2000 | Test Acc: 0.1000
Seed 30 | Epoch 007 | Loss: 1.7780 | Train Acc: 0.1979 | Test Acc: 0.1667
Seed 30 | Epoch 008 | Loss: 1.7756 | Train Acc: 0.2250 | Test Acc: 0.1500
Seed 30 | Epoch 009 | Loss: 1.7747 | Train Acc: 0.2250 | Test Acc: 0.1083
Seed 30 | Epoch 010 | Loss: 1.7708 | Train Acc: 0.2188 | Test Acc: 0.1583
Seed 30 | Epoch 011 | Loss: 1.7662 | Train Acc: 0.2437 | Test Acc: 0.1250
Seed 30 | Epoch 012 | Loss: 1.7611 | Train Acc: 0.2687 | Test Acc: 0.1750
Seed 30 | Epoch 013 | Loss: 1.7614 | Train Acc: 0.2292 | Test Acc: 0.1500
Seed 30 | Epoch 014 | Loss: 1.7608 | T

epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▇▇▆▆▆▆▆▆▆▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁
test_acc,▄▂▁▁▄▃▁▄▂▄▃▄▄▅▃▅▄▅▄▅▄▆▆▇▆▆▅▆▇▅▇▅█▄▆█▇▅▇▅
train_acc,▁▂▂▂▃▃▃▃▄▅▄▄▄▅▄▅▄▄▅▄▅▅▅▅▅▅▆▅▆▆▇▇▆▇▆▆█▆▇▇
epoch,50
loss,1.6409
test_acc,0.2
train_acc,0.32917


# **BFS and centrality**

In [6]:
import os, random, numpy as np, torch
import torch.nn.functional as F
from torch import nn
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_networkx
import networkx as nx
import wandb

EPOCHS = int(os.getenv("EPOCHS", "50"))
NUM_SEEDS = int(os.getenv("NUM_SEEDS", "30"))
PROJECT = f"GCN Graph classification({Dataset_name})_Gal"
ENTITY = "noynetanel1-tel-aviv-university"

def set_seed(s):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, cent_dim=16):
        super().__init__()
        self.cent_enc = nn.Linear(1, cent_dim)
        self.conv1 = GCNConv(in_channels + cent_dim, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = nn.Linear(hidden_channels, out_channels)
    def forward(self, x, cent2, edge_index, batch):
        if x is None:
            x = torch.ones((cent2.shape[0], 1), device=cent2.device)
        x = torch.cat([x, self.cent_enc(cent2.unsqueeze(1))], dim=1)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        return F.log_softmax(self.lin(x), dim=1)

def make_centrality_transform():
    def transform(data):
        G = to_networkx(data, to_undirected=True)
        n = data.num_nodes
        deg_cent = nx.degree_centrality(G) if G.number_of_nodes() > 0 else {}
        cent = torch.zeros((n,), dtype=torch.float)
        for u in range(n):
            cent[u] = float(deg_cent.get(u, 0.0))
        data.cent2 = cent
        if getattr(data, "x", None) is None:
            data.x = torch.ones((n, 1), dtype=torch.float)
        return data
    return transform

def run_seed(SEED):
    set_seed(SEED)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    dataset = TUDataset(root="data/TUDataset", name=Dataset_name, transform=make_centrality_transform())
    torch.manual_seed(SEED)
    dataset = dataset.shuffle()
    num_graphs = len(dataset)
    train_size = int(0.8 * num_graphs)
    train_dataset = dataset[:train_size]
    test_dataset = dataset[train_size:]
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

    wandb.init(
        project=PROJECT,
        entity=ENTITY,
        name=f"GCN_CentralityOnly_seed_{SEED}",
        group=f"GCN_{Dataset_name}_CentralityOnly",
        job_type="centrality-only",
        reinit=True,
        config={
            "model": "GCN",
            "hidden_channels": 64,
            "epochs": EPOCHS,
            "lr": 0.001,
            "dataset": Dataset_name,
            "task": "Centrality_Only",
            "seed": SEED,
            "cuda_available": torch.cuda.is_available(),
            "gpu_name": torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU",
            "cuda_version": torch.version.cuda if torch.cuda.is_available() else None
        }
    )

    model = GCN(
        in_channels=dataset.num_features if dataset.num_features else 1,
        hidden_channels=64,
        out_channels=dataset.num_classes,
        cent_dim=16
    ).to(device)

    opt = torch.optim.Adam(model.parameters(), lr=0.001)

    def train():
        model.train()
        total_loss = 0.0
        correct = 0
        total = 0
        for data in train_loader:
            data = data.to(device)
            out = model(data.x, data.cent2, data.edge_index, data.batch)
            loss = F.nll_loss(out, data.y)
            opt.zero_grad()
            loss.backward()
            opt.step()
            total_loss += loss.item() * data.num_graphs
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
            total += data.num_graphs
        return total_loss / len(train_dataset), correct / total

    def test():
        model.eval()
        correct = 0
        for data in test_loader:
            data = data.to(device)
            out = model(data.x, data.cent2, data.edge_index, data.batch)
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
        return correct / len(test_dataset)

    for epoch in range(1, EPOCHS + 1):
        loss, train_acc = train()
        test_acc = test()
        print(f"Seed {SEED} | Epoch {epoch:03d} | Loss: {loss:.4f} | Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f}")
        wandb.log({"epoch": epoch, "loss": loss, "train_acc": train_acc, "test_acc": test_acc})

    wandb.finish()

if __name__ == "__main__":
    wandb.login()
    for s in range(1, NUM_SEEDS + 1):
        run_seed(s)

Seed 1 | Epoch 001 | Loss: 1.7979 | Train Acc: 0.1521 | Test Acc: 0.1750
Seed 1 | Epoch 002 | Loss: 1.7900 | Train Acc: 0.1958 | Test Acc: 0.2250
Seed 1 | Epoch 003 | Loss: 1.7874 | Train Acc: 0.2062 | Test Acc: 0.1500
Seed 1 | Epoch 004 | Loss: 1.7838 | Train Acc: 0.1896 | Test Acc: 0.1750
Seed 1 | Epoch 005 | Loss: 1.7854 | Train Acc: 0.2104 | Test Acc: 0.2500
Seed 1 | Epoch 006 | Loss: 1.7782 | Train Acc: 0.2313 | Test Acc: 0.1333
Seed 1 | Epoch 007 | Loss: 1.7764 | Train Acc: 0.1750 | Test Acc: 0.1500
Seed 1 | Epoch 008 | Loss: 1.7722 | Train Acc: 0.1896 | Test Acc: 0.2083
Seed 1 | Epoch 009 | Loss: 1.7701 | Train Acc: 0.2021 | Test Acc: 0.2250
Seed 1 | Epoch 010 | Loss: 1.7658 | Train Acc: 0.1896 | Test Acc: 0.1833
Seed 1 | Epoch 011 | Loss: 1.7621 | Train Acc: 0.1979 | Test Acc: 0.1917
Seed 1 | Epoch 012 | Loss: 1.7627 | Train Acc: 0.2083 | Test Acc: 0.2000
Seed 1 | Epoch 013 | Loss: 1.7557 | Train Acc: 0.1938 | Test Acc: 0.1583
Seed 1 | Epoch 014 | Loss: 1.7551 | Train Acc: 0.20

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▇▇▇▆▆▆▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▂▃▂▂▂▂▂▂▁▁▁
test_acc,▂▅▁▂▇▁▄▅▃▃▁▂▂▅▆▅▁▅▅▃▄▆▅▄▁▇▆▄▅▃▃█▅▅▅▆▅▅▆▅
train_acc,▁▄▄▃▅▂▄▃▄▅▄▄▅▆▅▃▅▅▅▄▆▇▆▅▆▆▆▇▆▇▆▇█▆█▆▇▇▇▇
epoch,50
loss,1.70428
test_acc,0.21667
train_acc,0.24375


Seed 2 | Epoch 001 | Loss: 1.7976 | Train Acc: 0.1688 | Test Acc: 0.1750
Seed 2 | Epoch 002 | Loss: 1.7872 | Train Acc: 0.2000 | Test Acc: 0.1500
Seed 2 | Epoch 003 | Loss: 1.7836 | Train Acc: 0.2042 | Test Acc: 0.1833
Seed 2 | Epoch 004 | Loss: 1.7805 | Train Acc: 0.2167 | Test Acc: 0.1833
Seed 2 | Epoch 005 | Loss: 1.7781 | Train Acc: 0.2021 | Test Acc: 0.2083
Seed 2 | Epoch 006 | Loss: 1.7700 | Train Acc: 0.2500 | Test Acc: 0.2167
Seed 2 | Epoch 007 | Loss: 1.7735 | Train Acc: 0.2125 | Test Acc: 0.1583
Seed 2 | Epoch 008 | Loss: 1.7625 | Train Acc: 0.2292 | Test Acc: 0.1917
Seed 2 | Epoch 009 | Loss: 1.7612 | Train Acc: 0.2313 | Test Acc: 0.2167
Seed 2 | Epoch 010 | Loss: 1.7560 | Train Acc: 0.2479 | Test Acc: 0.1917
Seed 2 | Epoch 011 | Loss: 1.7541 | Train Acc: 0.2292 | Test Acc: 0.2083
Seed 2 | Epoch 012 | Loss: 1.7509 | Train Acc: 0.2458 | Test Acc: 0.2000
Seed 2 | Epoch 013 | Loss: 1.7454 | Train Acc: 0.2354 | Test Acc: 0.1917
Seed 2 | Epoch 014 | Loss: 1.7417 | Train Acc: 0.23

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▇▇▆▆▅▅▅▄▄▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁
test_acc,▂▁▃▃▄▁▃▅▃▄▃▆▆▅▄▅▆▄▅▅▅▆▆▅▅▅▆▅▆▅▄▇▆▅▇▇▇█▆▇
train_acc,▁▃▃▄▃▄▅▅▆▅▅▅▇▆▆▆▆▅▅▆▅▆▇▆▆▆▅▆▇▆▇▇▇▇▇▇█▇▇▆
epoch,50
loss,1.71418
test_acc,0.25833
train_acc,0.25208


Seed 3 | Epoch 001 | Loss: 1.8020 | Train Acc: 0.1437 | Test Acc: 0.2333
Seed 3 | Epoch 002 | Loss: 1.7923 | Train Acc: 0.1729 | Test Acc: 0.1583
Seed 3 | Epoch 003 | Loss: 1.7878 | Train Acc: 0.1750 | Test Acc: 0.1833
Seed 3 | Epoch 004 | Loss: 1.7846 | Train Acc: 0.1667 | Test Acc: 0.1917
Seed 3 | Epoch 005 | Loss: 1.7822 | Train Acc: 0.1729 | Test Acc: 0.3000
Seed 3 | Epoch 006 | Loss: 1.7765 | Train Acc: 0.2146 | Test Acc: 0.2167
Seed 3 | Epoch 007 | Loss: 1.7788 | Train Acc: 0.2042 | Test Acc: 0.2417
Seed 3 | Epoch 008 | Loss: 1.7734 | Train Acc: 0.2188 | Test Acc: 0.1667
Seed 3 | Epoch 009 | Loss: 1.7675 | Train Acc: 0.1917 | Test Acc: 0.2583
Seed 3 | Epoch 010 | Loss: 1.7700 | Train Acc: 0.1854 | Test Acc: 0.2333
Seed 3 | Epoch 011 | Loss: 1.7651 | Train Acc: 0.2146 | Test Acc: 0.2833
Seed 3 | Epoch 012 | Loss: 1.7661 | Train Acc: 0.2229 | Test Acc: 0.2083
Seed 3 | Epoch 013 | Loss: 1.7570 | Train Acc: 0.2042 | Test Acc: 0.2917
Seed 3 | Epoch 014 | Loss: 1.7578 | Train Acc: 0.18

...

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▇▇▇▆▆▅▅▅▄▄▃▄▃▃▃▃▃▃▂▂▃▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁
test_acc,▂▁▅▃▆▃▃▅▇▃▅▆▅▅▆▅▄▄▅▆▄▆▆▃▆▆▆▆▃▅▆▇▆█▇▇▇▆▆▇
train_acc,▁▂▁▂▃▄▄▅▄▆▆▆▅▄▆▄▅▆▆▅▆▆▆▅▆▆▅▆▇▆▆▅▇▆▇█▇▆▇▇
epoch,50
loss,1.69875
test_acc,0.21667
train_acc,0.30417


Seed 29 | Epoch 001 | Loss: 1.7984 | Train Acc: 0.1354 | Test Acc: 0.1917
Seed 29 | Epoch 002 | Loss: 1.7902 | Train Acc: 0.1729 | Test Acc: 0.1833
Seed 29 | Epoch 003 | Loss: 1.7862 | Train Acc: 0.2333 | Test Acc: 0.1750
Seed 29 | Epoch 004 | Loss: 1.7846 | Train Acc: 0.2125 | Test Acc: 0.1667
Seed 29 | Epoch 005 | Loss: 1.7794 | Train Acc: 0.2021 | Test Acc: 0.1917
Seed 29 | Epoch 006 | Loss: 1.7820 | Train Acc: 0.2125 | Test Acc: 0.1750
Seed 29 | Epoch 007 | Loss: 1.7761 | Train Acc: 0.2167 | Test Acc: 0.1750
Seed 29 | Epoch 008 | Loss: 1.7722 | Train Acc: 0.2000 | Test Acc: 0.2583
Seed 29 | Epoch 009 | Loss: 1.7697 | Train Acc: 0.2313 | Test Acc: 0.2000
Seed 29 | Epoch 010 | Loss: 1.7667 | Train Acc: 0.2083 | Test Acc: 0.2333
Seed 29 | Epoch 011 | Loss: 1.7666 | Train Acc: 0.2313 | Test Acc: 0.2000
Seed 29 | Epoch 012 | Loss: 1.7584 | Train Acc: 0.2396 | Test Acc: 0.1917
Seed 29 | Epoch 013 | Loss: 1.7574 | Train Acc: 0.2167 | Test Acc: 0.2083
Seed 29 | Epoch 014 | Loss: 1.7583 | T

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▇▇▆▆▆▆▅▅▅▄▄▄▄▄▄▃▄▄▃▃▃▃▃▃▂▂▃▂▂▂▂▂▂▁▁▂▁▁
test_acc,▂▂▁▁▂▆▃▄▃▂▃▃▇▃▆▄▅▆▆▆▅▄▆▄▇▆▇▇▇▇█▆▆▅▇▇▄▆▇█
train_acc,▁▃▆▅▄▅▄▆▅▆▅▆▆▆▇▇▆▆█▇▆▇▇▇▇▇▇▇▆█▇▇▇▇▆▇█▇▆▇
epoch,50
loss,1.71558
test_acc,0.29167
train_acc,0.2625


Seed 30 | Epoch 001 | Loss: 1.7990 | Train Acc: 0.1667 | Test Acc: 0.1333
Seed 30 | Epoch 002 | Loss: 1.7876 | Train Acc: 0.1958 | Test Acc: 0.1417
Seed 30 | Epoch 003 | Loss: 1.7840 | Train Acc: 0.2104 | Test Acc: 0.1500
Seed 30 | Epoch 004 | Loss: 1.7811 | Train Acc: 0.1875 | Test Acc: 0.1833
Seed 30 | Epoch 005 | Loss: 1.7764 | Train Acc: 0.2083 | Test Acc: 0.2000
Seed 30 | Epoch 006 | Loss: 1.7720 | Train Acc: 0.2250 | Test Acc: 0.1667
Seed 30 | Epoch 007 | Loss: 1.7707 | Train Acc: 0.2313 | Test Acc: 0.1917
Seed 30 | Epoch 008 | Loss: 1.7704 | Train Acc: 0.2208 | Test Acc: 0.2583
Seed 30 | Epoch 009 | Loss: 1.7619 | Train Acc: 0.2188 | Test Acc: 0.1833
Seed 30 | Epoch 010 | Loss: 1.7601 | Train Acc: 0.2188 | Test Acc: 0.2583
Seed 30 | Epoch 011 | Loss: 1.7553 | Train Acc: 0.2167 | Test Acc: 0.2250
Seed 30 | Epoch 012 | Loss: 1.7511 | Train Acc: 0.2375 | Test Acc: 0.2417
Seed 30 | Epoch 013 | Loss: 1.7491 | Train Acc: 0.2479 | Test Acc: 0.2250
Seed 30 | Epoch 014 | Loss: 1.7506 | T

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▇▇▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁
test_acc,▁▁▃▄▂▇▃▇▅▆▅▇█▇▆▇█▇▇▅▇▇▆▇▆▆▆▆▇▆▅▆▆▇▇▇▆█▆▆
train_acc,▁▃▄▂▃▄▄▄▄▅▅▇▆▆▅▆▆▆▇▇▇▇▅▆▇▇▆▇▇▇▆▆▇▇▇█▆█▇▇
epoch,50
loss,1.69906
test_acc,0.23333
train_acc,0.26875
